In [1]:
import glob

import numpy
import torch
import PIL.Image
import torchvision.tv_tensors
import torchvision.transforms.v2

transforms = torchvision.transforms.v2.Compose([
    torchvision.transforms.v2.CenterCrop((224, 224)),
    torchvision.transforms.v2.Resize((64, 64)),
    torchvision.transforms.v2.ToDtype(torch.float32, scale = True)
])
def transforms_wrapper(image, mask):
    image = torchvision.tv_tensors.Image(image)
    mask = torchvision.tv_tensors.Mask(mask)
    return transforms(image, mask)

def load_dataset(path):
    images, masks = [], []
    for filename in glob.glob(f"{path}/image/*"):
        image, mask = transforms_wrapper(
            PIL.Image.open(filename),
            PIL.Image.open(filename.replace("image", "mask").replace("jpg", "png"))
        )
        images.append(image.permute((1, 2, 0)).numpy())
        masks.append(mask.squeeze().numpy())
    return numpy.array(images)[:75, :, :, 0], numpy.array(masks)[:75]

train_images, train_masks = load_dataset("midv500p/train")
test_images, test_masks = load_dataset("midv500p/val")

In [2]:
import cvtda.topology
extractor = cvtda.topology.FeatureExtractor(
    n_jobs = 1,
    binarizer_thresholds = [ 0.4 ],
    num_radial_filtrations = 2
)
train_features = extractor.fit_transform(train_images, "seg_train")
test_features = extractor.transform(test_images, "seg_test")

Calculating inverted images
GreyscaleExtractor: processing seg_train/greyscale, do_fit = True
Saving the result to ./seg_train/greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
DiagramVectorizer: fitting complete


DiagramVectorizer: batch: 100%|██████████| 1/1 [00:00<00:00,  9.11it/s]

Saving the result to ./seg_train/greyscale/features.npy
GreyscaleExtractor: processing seg_train/greyscale, do_fit = False
Got the result from ./seg_train/greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
Got the result from ./seg_train/greyscale/features.npy
GreyscaleExtractor: processing seg_train/inverted_greyscale, do_fit = True


Saving the result to ./seg_train/inverted_greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
DiagramVectorizer: fitting complete


DiagramVectorizer: batch: 100%|██████████| 1/1 [00:00<00:00, 10.75it/s]

...

Saving the result to ./seg_test/filtrations/4/RadialFiltration_43_43/diagrams.npy
Applying Scaler to persistence diagrams.


DiagramVectorizer: batch: 100%|██████████| 1/1 [00:00<00:00, 42.54it/s]


Saving the result to ./seg_test/filtrations/4/RadialFiltration_43_43/features.npy


GrayGeometryExtractor: 100%|██████████| 62/62 [00:06<00:00,  9.80it/s]

Saving the result to ./seg_test/geometry/features.npy
Applying StandardScaler.


In [3]:
import cvtda.segmentation
cvtda.segmentation.segment(
    train_images, train_features, train_masks,
    test_images, test_features, test_masks,
    n_epochs = 3
)

Trying no - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x000002CC8E0B1B10>
Parameters: 26435305


100%|██████████| 3/3 [00:00<00:00,  3.67it/s, loss=2.64, lr=1e-5, Accuracy=0.562, AUC-ROC=0.662, Precision=0, Recall=0, F1-score=0, Jaccard score=0]  


{'model': 'Без признаков', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.6616421770884784, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}
Trying images - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x000002CC8E0B1C30>
Parameters: 26435305


100%|██████████| 3/3 [00:00<00:00,  4.38it/s, loss=2.64, lr=1e-5, Accuracy=0.562, AUC-ROC=0.662, Precision=0, Recall=0, F1-score=0, Jaccard score=0]  


{'model': 'Без топологических признаков', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.6616088111265003, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}
Trying topological - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x000002CC8E0B2050>
Parameters: 26435305


100%|██████████| 3/3 [00:00<00:00,  4.01it/s, loss=2.64, lr=1e-5, Accuracy=0.562, AUC-ROC=0.661, Precision=0, Recall=0, F1-score=0, Jaccard score=0] 


{'model': 'Только топологические признаки', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.6611359397225922, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}
Trying combined - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x000002CC8E0B1C90>
Parameters: 26509033


100%|██████████| 3/3 [00:00<00:00,  3.97it/s, loss=2.7, lr=1e-5, Accuracy=0.562, AUC-ROC=0.67, Precision=0, Recall=0, F1-score=0, Jaccard score=0]    


{'model': 'Комбинированные признаки', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.6702702122935295, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}


,model,Accuracy,AUC-ROC,Precision,Recall,F1-score,Jaccard score
0,Без признаков,0.562087,0.661642,0.0,0.0,0.0,0.0
1,Без топологических признаков,0.562087,0.661609,0.0,0.0,0.0,0.0
2,Только топологические признаки,0.562087,0.661136,0.0,0.0,0.0,0.0
3,Комбинированные признаки,0.562087,0.670270,0.0,0.0,0.0,0.0
